In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
import pandas as pd

In [2]:
model = TypePredictor()
model.fit(number=10)

In [3]:
dataset = SimpleDatasetGenerator()

df, y = dataset.generate_simple_dataframe(mix_codes=False)

model.predict_dataframe(df)

,CodiceFiscale,IVA,Address,CAP,NaN,ProvinceCode,Region,Municipality,Year,Integer,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Currency,Date,HouseNumber,BiologicalSex
0,CodiceFiscale,IVA,Address,CAP,NaN,ProvinceCode,Region,Municipality,Year,Integer,...,CountryCode,Name,Surname,String,NaN,NaN,Currency,Date,NaN,NaN
1,CodiceFiscale,IVA,Address,CAP,NaN,ProvinceCode,NaN,Municipality,Year,Integer,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Currency,Date,HouseNumber,BiologicalSex
2,CodiceFiscale,IVA,Address,CAP,NaN,ProvinceCode,Region,Municipality,Year,Integer,...,CountryCode,Name,NaN,String,EMail,NaN,Currency,Date,HouseNumber,BiologicalSex
3,CodiceFiscale,IVA,Address,NaN,NaN,ProvinceCode,NaN,Municipality,Year,Integer,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Currency,Date,NaN,BiologicalSex
4,CodiceFiscale,IVA,Address,CAP,NaN,ProvinceCode,Region,Municipality,Year,Integer,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Currency,Date,HouseNumber,NaN
5,CodiceFiscale,IVA,Address,CAP,NaN,ProvinceCode,Region,NaN,Year,Integer,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Currency,Date,HouseNumber,BiologicalSex
6,CodiceFiscale,IVA,NaN,CAP,NaN,ProvinceCode,Region,Municipality,Year,Integer,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Currency,NaN,HouseNumber,BiologicalSex
7,CodiceFiscale,IVA,Address,CAP,NaN,ProvinceCode,Region,Municipality,Year,Integer,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Currency,Date,HouseNumber,BiologicalSex
8,CodiceFiscale,IVA,Address,CAP,NaN,ProvinceCode,Region,Municipality,NaN,Integer,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Currency,Date,HouseNumber,BiologicalSex
9,CodiceFiscale,NaN,Address,CAP,NaN,ProvinceCode,Region,Municipality,Year,Integer,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,NaN,Date,HouseNumber,BiologicalSex


In [4]:
df

,CodiceFiscale,IVA,Address,CAP,NaN,ProvinceCode,Region,Municipality,Year,Integer,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Currency,Date,HouseNumber,BiologicalSex
0,FRLRFL45C09L706C,00310750849,Via Delle Colombaie,36043,.,SO,VENETO,DORMELLETTO,2001.0,-91351.0,...,KR,valoree,hinkle,Scusami!/ Mi scusi! (polite),NaN,NaN,"396,00 €",1936-12-12,NaN,NaN
1,PCNSVN26P44F572T,04031030879,Via G. B. Giuliani,62011,_,MC,NaN,FAEDIS,2012.0,92567.0,...,ES,cathrien,golab,il ristorante,audry.yaw@yaw.org,+39-377-555-8970,"50.625,00 €",1935-10-19,121/2,donna
2,FRRGRL68H09D175R,10409911004,Via Nazionale Jonica,22066,_,PI,SICILIA,ROVOLON,2000.0,20560.0,...,VE,maëly,NaN,la primavera prossima,golda_kaniecki@yahoo.com,NaN,"41.257,00 €",1985-07-21,9/10,uomo
3,GBRRCR63R17A437G,01510400805,Via Val D'enza,NaN,NaN,TE,NaN,VILLAMAR,1998.0,40102.0,...,DE,peabody,danner,la nuora,leonora@yahoo.com,+39-373-555-984385,"27.770,00 €",1991-06-09,NaN,donna
4,RSSLCN85D52B288L,06537350636,Via Seminario Estivo - Is.10 Pal.b,85047,.,CB,BASILICATA,SERRA DE' CONTI,1998.0,-83845.0,...,CU,zackeria,sepulveda,Dove vivi?/ Dove vive? (polite),dyum@yahoo.com,+39-351-055-528670,"30.794,00 €",1975-07-10,279,NaN
5,RSSDNS87D64F507G,04553790728,Viale Di Ponente,57128,\n,BS,PIEMONTE,NaN,2026.0,-99668.0,...,BT,alizza,vizena,la ruota,scarlet.cartan@yahoo.com,+39-368-555-077,"31.647,00 €",1980-02-13,109/113,F
6,LOIMNM21B42I668O,01152260806,NaN,12074.0,.,PT,LIGURIA,MONTE ROMANO,2027.0,NaN,...,NG,darrilyn,latz,Sorrido,carmelina_lindall@lindall.com,+39-370-555-882,"98.695,00 €",NaN,26-28-30,uomo
7,CSTFNC78H28L040O,02683600544,Piazza Santa Maria Novella,37121,0,BR,PUGLIA,VILLAFRATI,1994.0,45771.0,...,JM,dunstan,bellar,la gonna,nana@aol.com,+39-377-555-523,"78.645,00 €",1976-12-02,20 A,maschio
8,CRTMRA76M59B178U,02116940426,Via Crociata,50026,/,PU,LAZIO,CEPPO MORELLI,NaN,93220.0,...,GN,jakey,doli,cinque,erinn.canlas@canlas.com,+39-351-255-599331,"89.897,00 €",1981-03-21,183/187,uomo
9,RSSMLN81C60E394Y,NaN,Via Duca Del Mare,15059.0,NaN,BL,PIEMONTE,CASTELFRANCO IN MISCANO,1995.0,98624.0,...,SK,oys,lantier,l'ingegnere,freeman_gochal@aol.com,+39-351-055-5274,NaN,1944-12-14,84/B,uomo
